In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
import re 
from keras.preprocessing.text import Tokenizer 

In [ ]:
# Mount the google drive 
from google.colab import drive
drive.mount('/content/drive')
#''' Filepath : AP '''
#train_filepath = "/content/drive/MyDrive/datasets/CNNDAILYMAIL/cnn_dailymail/train.csv"
#test_filepath = "/content/drive/MyDrive/datasets/CNNDAILYMAIL/cnn_dailymail/test.csv"
#validation_filepath = "/content/drive/MyDrive/datasets/CNNDAILYMAIL/cnn_dailymail/validation.csv"
#''' FilePath: SN'''
train_filepath = "/content/drive/MyDrive/Colab Notebooks/Fuse_project/CNN_dailymail/train.csv"
test_filepath = "/content/drive/MyDrive/Colab Notebooks/Fuse_project/CNN_dailymail/test.csv"
validation_filepath = "/content/drive/MyDrive/Colab Notebooks/Fuse_project/CNN_dailymail/validation.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv(train_filepath)
test_df = pd.read_csv(test_filepath)
validation_df = pd.read_csv(validation_filepath)

### Explore the data 


In [ ]:
print(f"train_df.shape: {train_df.shape}")
print(f"test_df.shape: {test_df.shape}")
print(f"valid_df.shape: {validation_df.shape}")

train_df.shape: (287113, 3)
test_df.shape: (11490, 3)
valid_df.shape: (13368, 3)


In [ ]:
train_df.head(10)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
5,0004306354494f090ee2d7bc5ddbf80b63e80de6,He's been accused of making many a fashion fau...,Prime Minister and his family are enjoying an ...
6,0005d61497d21ff37a17751829bd7e3b6e4a7c5c,By . Daily Mail Reporter . PUBLISHED: . 01:15 ...,NBA star calls for black and Hispanic communit...
7,0006021f772fad0aa78a977ce4a31b3faa6e6fe5,By . Daily Mail Reporter . This is the moment ...,London Midland service had been pulling into T...
8,00083697263e215e5e7eda753070f08aa374dd45,There are a number of job descriptions waiting...,Tony Pulis believes Saido Berahino should look...
9,000940f2bb357ac04a236a232156d8b9b18d1667,"Canberra, Australia (CNN) -- At first glance, ...",Black box data from Flight 370 could be analyz...


### Data Preprocessing
#### Check null values in the data  

In [ ]:
print(train_df.isna().sum())
print(test_df.isna().sum())
print(validation_df.isna().sum())

id            0
article       0
highlights    0
dtype: int64
id            0
article       0
highlights    0
dtype: int64
id            0
article       0
highlights    0
dtype: int64


#### Check duplicates in the data 

In [ ]:
print(train_df.duplicated(subset = ["article", "highlights"]).sum())
print(test_df.duplicated(subset = ["article", "highlights"]).sum())
print(validation_df.duplicated(subset = ["article", "highlights"]).sum())

3098
2
0


#### Drop duplicates

In [ ]:
train_df = train_df.drop_duplicates(subset= ['article', 'highlights'])
test_df = test_df.drop_duplicates(subset= ['article', 'highlights'])
validation_df = validation_df.drop_duplicates(subset= ['article', 'highlights'])

In [ ]:
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [ ]:
train_df = train_df.drop(["id"], axis = 1)
test_df = test_df.drop(["id"], axis = 1)
validation_df = validation_df.drop(["id"], axis = 1)

In [ ]:
validation_df.head(10)

,article,highlights
0,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...
5,The team with the lowest transfer outlay this ...,Stoke City beat Everton 2-0 to move up to eigh...
6,A team from a Spanish university . has discove...,"Woman was an aristocrat from Elephantine, coun..."
7,A doctor who made a stand against sexism in me...,"Dr Ashleigh Witt, from Melbourne, wrote essay ..."
8,Ronda Rousey recorded the fastest-ever finish ...,Ronda Rousey submitted Cat Zingano via armbar ...
9,Celtic defender Virgil van Dijk admits he fear...,Celtic defeated Dundee United 2-0 in Scottish ...


#### Preprocessing 
1. Convert every text to lower case 
2. Remove HTML tags
3. Contraction mapping
4. Remove (‘s)
5. Remove any text inside the parenthesis ( )
6. Eliminate punctuations and special characters
7. Remove stopwords
8. Remove short words


In [ ]:
#Download stopwords and store them in stop_words 
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

"didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

"he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

"I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

"i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

"mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

"mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
"she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

"should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

"this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

"there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

"they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

"we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
"what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

"where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

"why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

"you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

"you're": "you are", "you've": "you have"}

In [ ]:
# Clean article from the data 
def text_cleaner(text):
  new_text = text.lower()
  new_text = BeautifulSoup(new_text, "lxml").text
  new_text = re.sub(r'\([^)]*\)', '', new_text)
  new_text = re.sub('"','', new_text)
  new_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_text.split(" ")]) 
  new_text = re.sub(r"'s\b","",new_text)
  new_text = re.sub("[^a-zA-Z]", " ", new_text)
  tokens = [w for w in new_text.split() if not w in stop_words]

  long_words=[]
  for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
  return (" ".join(long_words)).strip()

cleaned_train_text = []
cleaned_test_text = []
cleaned_validation_text = []

for t in train_df['article']:
    cleaned_train_text.append(text_cleaner(t))

for t in test_df['article']:
    cleaned_test_text.append(text_cleaner(t))

for t in validation_df['article']:
  cleaned_validation_text.append(text_cleaner(t))

In [ ]:
cleaned_train_text[:1]

['associated press published est october updated est october bishop fargo catholic diocese north dakota exposed potentially hundreds church members fargo grand forks jamestown hepatitis virus late september early october state health department issued advisory exposure anyone attended five churches took communion bishop john folda fargo catholic diocese north dakota exposed potentially hundreds church members fargo grand forks jamestown hepatitis state immunization program manager molly howell says risk low officials feel important alert people possible exposure diocese announced monday bishop john folda taking time diagnosed hepatitis diocese says contracted infection contaminated food attending conference newly ordained bishops italy last month symptoms hepatitis include fever tiredness loss appetite nausea abdominal discomfort fargo catholic diocese north dakota bishop located']

In [ ]:
# Clean highlights from the data 
def summary_cleaner(text):
    new_text = re.sub('"','', text)
    new_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in new_text.split(" ")])    
    new_text = re.sub(r"'s\b","",new_text)
    new_text = re.sub("[^a-zA-Z]", " ", new_text)
    new_text = new_text.lower()
    tokens= new_text.split()
    new_text=''
    for i in tokens:
        if len(i)>1:                                 
            new_text = new_text+i+' '  
    return new_text

#Call the above function
cleaned_train_highligts = []
cleaned_test_highlights = []
cleaned_validation_hightlights = []

for t in train_df['highlights']:
    cleaned_train_highligts.append(summary_cleaner(t))

for t in test_df['highlights']:
    cleaned_test_highlights.append(summary_cleaner(t))

for t in validation_df['highlights']:
    cleaned_validation_hightlights.append(summary_cleaner(t))



train_df['cleaned_article']= cleaned_train_text
test_df['cleaned_article'] = cleaned_test_text
validation_df['cleaned_article'] = cleaned_validation_text


train_df['cleaned_highlights'] = cleaned_train_highligts
test_df['cleaned_highlights'] = cleaned_test_highlights
validation_df['cleaned_highlights'] = cleaned_validation_hightlights


train_df['cleaned_highlights'].replace('', np.nan, inplace=True)
train_df.dropna(axis=0,inplace=True)

test_df['cleaned_highlights'].replace('', np.nan, inplace=True)
test_df.dropna(axis=0,inplace=True)

validation_df['cleaned_highlights'].replace('', np.nan, inplace=True)
validation_df.dropna(axis=0,inplace=True)


In [ ]:
train_df.head()

,article,highlights,cleaned_article,cleaned_highlights
0,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",associated press published est october updated...,bishop john folda of north dakota is taking ti...
1,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...,ralph mata internal affairs lieutenant miami d...,criminal complaint cop used his role to help c...
2,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t...",drunk driver killed young woman head crash che...,craig eccleston todd had drunk at least three ...
3,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...,breezy sweep pen president vladimir putin wrot...,nina dos santos says europe must be ready to a...
4,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...,fleetwood team still record sky bet league one...,fleetwood top of league one after win at scunt...


In [ ]:
for i in range(2):
    print("Review:",train_df['cleaned_article'][i])
    print("Summary:",train_df['cleaned_highlights'][i])
    print("\n")

Review: associated press published est october updated est october bishop fargo catholic diocese north dakota exposed potentially hundreds church members fargo grand forks jamestown hepatitis virus late september early october state health department issued advisory exposure anyone attended five churches took communion bishop john folda fargo catholic diocese north dakota exposed potentially hundreds church members fargo grand forks jamestown hepatitis state immunization program manager molly howell says risk low officials feel important alert people possible exposure diocese announced monday bishop john folda taking time diagnosed hepatitis diocese says contracted infection contaminated food attending conference newly ordained bishops italy last month symptoms hepatitis include fever tiredness loss appetite nausea abdominal discomfort fargo catholic diocese north dakota bishop located
Summary: bishop john folda of north dakota is taking time off after being diagnosed he contracted the

# Model Building


In [ ]:
import numpy as np
import pandas as pd

Installing the dependencies

In [ ]:
!pip install --quiet transformers==4.5.0
# !pip install --quiet pytorch-lightning

In [ ]:
!pip install --upgrade pytorch-lightning

In [ ]:
import json 
import torch
from pathlib import Path
from torch.utils.data import Dataset , DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger


In [ ]:
class NewsSummaryDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        text_max_token_len: int = 512,
        summary_max_token_len: int = 128
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == 0] = -100 # to make sure we have correct labels for T5 text generation

        return dict(
            text=text,
            summary=data_row['summary'],
            text_input_ids=text_encoding['input_ids'].flatten(),
            text_attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            labels_attention_mask=summary_encoding['attention_mask'].flatten()
        )
     

In [ ]:
class NewsSummaryDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        text_max_token_len: int = 512,
        summary_max_token_len: int = 128
    ):
        super().__init__()

        self.train_df = train_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len

    def setup(self, stage=None):
        self.train_dataset = NewsSummaryDataset(
            self.train_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )
        self.test_dataset = NewsSummaryDataset(
            self.test_df,
            self.tokenizer,
            self.text_max_token_len,
            self.summary_max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2
        )

    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=2
        )
     

In [ ]:
MODEL_NAME = 't5-base'

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
     

In [ ]:
text_token_counts, summary_token_counts = [], []

for _, row in train_df.iterrows():
    text_token_count = len(tokenizer.encode(row['text']))
    text_token_counts.append(text_token_count)

    summary_token_count = len(tokenizer.encode(row['summary']))
    summary_token_counts.append(summary_token_count) 

In [ ]:
N_EPOCHS = 3
BATCH_SIZE = 8

data_module = NewsSummaryDataModule(train_df, test_df, tokenizer)

In [ ]:
class NewsSummaryModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
    
    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )

        return output.loss, output.logits

    def training_step(self, batch, batch_size):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_size):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_size):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=labels_attention_mask,
            labels=labels
        )

        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)
     


In [ ]:
model = NewsSummaryModel()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

logger = TensorBoardLogger("lightning_logs", name='news-summary')

trainer = pl.Trainer(
    logger=logger,
    checkpoint_callback=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate=30
)
     

In [ ]:
trained_model = NewsSummaryModel.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path
)
trained_model.freeze()

In [ ]:
def summarizeText(text):
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = trained_model.model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)
     

In [ ]:

sample_row = test_df.iloc[0]
text = sample_row['text']
model_summary = summarizeText(text)
text

In [ ]:
sample_row['summary']

In [ ]:
model_summary

In [ ]:

sample_row = test_df.iloc[1]
text = sample_row['text']
model_summary = summarizeText(text)
text
     

In [ ]:
sample_row['summary']

In [ ]:

model_summary

In [ ]:
model_summary = summarizeText("""Representative image
Representative image
ICICI Lombard
Trade
WatchlistPortfolioMessageSet Alert
 
 
NSELIVE
16 Apr, 2021 15:58
 1,417.15  30.55 (2.20%)
Volume 825956 Todays L/H 1,375.001,428.00
More 
Private general insurer, ICICI Lombard General Insurance posted a 22.6 percent year-on-year (YoY) rise in its March quarter (Q4) net profit at Rs 345.68 crore.

The general insurer had collected gross direct premium income (GDPI) of Rs 3,478 crore for Q4, a growth of 9.4 percent YoY.

The return on average equity (ROAE) was 21.7 percent in FY21 compared to 20.8 percent in FY20. ROAE was 18.8 percent in Q4FY21 compared to 18.8 percent in Q4FY20.

Solvency ratio was 2.90 at March 31, 2021 as against 2.76 at December 31, 2020. This was higher than the minimum regulatory requirement of 1.50. Solvency ratio was 2.17x at March 31, 2020.

As far as the COVID-19 health claims are concerned, general insurers saw 9,80,000 claims being filed worth Rs 14,000 crore.""")
model_summary

In [ ]:
import pickle

In [ ]:
filename = open('text_summarization_model.pkl', 'wb')
pickle.dump(trained_model.model, filename)

In [ ]:

def summarizeText(text):
    text_encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)